In [31]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
from timeit import default_timer as timer
from _collections import OrderedDict
import shutil
from pprint import pprint
start = timer()
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.file_location import FileLocationManager
from utilities.sqlcontroller import SqlController
from utilities.utilities_cvat_neuroglancer import get_structure_number, NumpyToNeuroglancer, get_segment_properties
from utilities.utilities_affine import align_point_sets, align_atlas, DK52_centers

In [3]:
animal = 'DK52'
fileLocationManager = FileLocationManager(animal)
OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'atlas')

In [6]:
sql_controller = SqlController(animal)
thumbnail_dir = os.path.join(fileLocationManager.prep, 'CH1', 'thumbnail_aligned')

print(f'resolution: {sql_controller.scan_run.resolution}')
print(f'width: {sql_controller.scan_run.width}')
print(f'height: {sql_controller.scan_run.height}')
box_w = sql_controller.scan_run.width * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_h = sql_controller.scan_run.height * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_z = len(os.listdir(thumbnail_dir))  # 20 mum scale
box_center = np.array([box_w, box_h, box_z]) / 2

atlasV7_volume = np.zeros((int(box_h), int(box_w), int(box_z)), dtype=np.uint8)
print('Shape of atlas volume', atlasV7_volume.shape)


resolution: 0.325
width: 65000
height: 36000
Shape of atlas volume (1170, 2112, 486)


In [43]:
reference_centers = sql_controller.get_centers_dict('DK52')

In [44]:
reference_centers

{'12N': [46488.0, 18778.0, 242.0],
 '5N_L': [38990.0, 20019.0, 172.0],
 '5N_R': [39184.0, 19027.0, 315.0],
 '7N_L': [42425.0, 23190.0, 166.0],
 '7N_R': [42286.0, 22901.0, 291.0]}

In [49]:
%%time
reference_centers = sql_controller.get_centers_dict('MD589')
atlas_box_size=(2010,1490,  447)
align_atlas(reference_centers, reference_scales=(0.46, 0.46,20), atlas_box_size=atlas_box_size)
#sql_controller.get_centers_dict('Atlas')

No histology for Atlas
No scan run for Atlas
Please type the following numbers into neuroglancer to align the atlas structures.

Rotation:
[[ 0.98616112 -0.08494664 -0.00192455]
 [ 0.08454767  0.98325432 -0.07613169]
 [ 0.00844547  0.07568627  0.98688077]]

Translation:
[[6743.21333857]
 [-925.71048559]
 [ -23.96175028]]
CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 128 ms


(array([[ 0.98616112, -0.08494664, -0.00192455],
        [ 0.08454767,  0.98325432, -0.07613169],
        [ 0.00844547,  0.07568627,  0.98688077]]),
 array([[3101.87813574],
        [-425.82682337],
        [-479.23500561]]))